In [10]:
import community 
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/kevingmagana/DSI/capstone/capstone-update/Capstone_Proposal/py_scripts/')
import find_citation
from graph_pipeline import GraphRecommender
import networkx as nx
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from collections import defaultdict
import copy
import re
import _pickle as cPickle



In [7]:
pwd

'/Users/kevingmagana/DSI/capstone/capstone-update/Capstone_Proposal/notebooks'

In [2]:
df = pd.read_csv('/Users/kevingmagana/DSI/capstone/case_corpus.csv')

In [3]:
data = df.case_text
corpus = df.drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1)
corpus = corpus.drop_duplicates(subset='title_date')
corpus = corpus.reset_index(drop=True)
# print(corpus.describe())
corpus.head()

,case_text,case_title,court,date,docket,tags,type_of_law,url,web_source,html_format,title_date
0,"United States Court of Appeals,Tenth Circuit.\...",Porro v. Barnes,United States Tenth Circuit,11/09/2010,10-6002,"Civil Rights, Constitutional Law, Immigration ...",case,http://caselaw.findlaw.com/us-10th-circuit/154...,findlaw.com,"[<div class=""caselawcontent searchable-content...",Porro v. Barnes 11/09/2010
1,"United States Court of Appeals,Tenth Circuit.\...",Garcia-Carbajal v. Holder,United States Tenth Circuit,11/05/2010,09-9558,"Administrative Law, Immigration Law",case,http://caselaw.findlaw.com/us-10th-circuit/154...,findlaw.com,"[<div class=""caselawcontent searchable-content...",Garcia-Carbajal v. Holder 11/05/2010
2,"United States Court of Appeals,Tenth Circuit.\...",US v. Rendon-Alamo,United States Tenth Circuit,10/19/2010,10-2089,"Sentencing, Immigration Law, Criminal Law & Pr...",case,http://caselaw.findlaw.com/us-10th-circuit/154...,findlaw.com,"[<div class=""caselawcontent searchable-content...",US v. Rendon-Alamo 10/19/2010
3,"United States Court of Appeals,Tenth Circuit.\...",Iliev v. Holder,United States Tenth Circuit,07/19/2010,09-9517,"Administrative Law, Immigration Law",case,http://caselaw.findlaw.com/us-10th-circuit/153...,findlaw.com,"[<div class=""caselawcontent searchable-content...",Iliev v. Holder 07/19/2010
4,"United States Court of Appeals,Tenth Circuit.\...",US v. Adame-Orozco,United States Tenth Circuit,06/04/2010,09-3296,"Criminal Law & Procedure, Immigration Law",case,http://caselaw.findlaw.com/us-10th-circuit/152...,findlaw.com,"[<div class=""caselawcontent searchable-content...",US v. Adame-Orozco 06/04/2010


## Step 1: Load up model (5-8 mins) and find community number for single court case

In [4]:
G = nx.Graph()

gr = GraphRecommender(corpus, G)
search_query='Carpio v. Holder 01/13/2010'
community_value = gr.parts[search_query]
print(community_value)

3


# Step 2: Return that case's community

In [5]:
commuity_nodes = gr.check[community_value]
print(len(commuity_nodes))
print(commuity_nodes[:10]) 


524
['Carpio v. Holder 01/13/2010', 'N-A-M v. Holder 11/20/2009', 'Mendiola v. Holder 10/28/2009', 'Rosillo-Puga v. Holder 09/15/2009', 'Herrera-Castillo v. Holder 07/27/2009', 'Wei v. Mukasey 11/10/2008', 'Lee v. Mukasey 06/03/2008', 'US v. Ochoa-Colchado 04/14/2008', 'Ochieng v. Mukasey 03/18/2008', 'Lorenzo v. Mukasey 11/21/2007']


## Step 3: Get Recommendations

In [8]:
gr.get_recommendations('Carpio v. Holder 01/13/2010', commuity_nodes)

['De Leon-Ochoa v. Attorney General 10/01/2010',
 'Quinchia v. US Atty. Gen. 08/07/2008',
 'Quinchia v. U.S. Attorney Gen. 12/18/2008',
 'Barrios v. Holder 05/27/2009',
 'Joseph v. Holder 08/27/2009']

## Save / Pickle the fitted graph model 

In [11]:
with open("/Users/kevingmagana/DSI/capstone/fitted_graph_model.pkl", "wb") as f:
    cPickle.dump(gr, f)


## Load the pickled graph model

In [12]:
with open("/Users/kevingmagana/DSI/capstone/fitted_graph_model.pkl", "rb") as f :
    model = cPickle.load(f)
    

## Test the (graph recommendations) model

In [16]:
search_query='Carpio v. Holder 01/13/2010'
community_value = model.parts[search_query]
commuity_nodes = model.check[community_value]
model.get_recommendations(search_query, commuity_nodes)

['De Leon-Ochoa v. Attorney General 10/01/2010',
 'Quinchia v. US Atty. Gen. 08/07/2008',
 'Quinchia v. U.S. Attorney Gen. 12/18/2008',
 'Barrios v. Holder 05/27/2009',
 'Joseph v. Holder 08/27/2009']

In [15]:
class GraphRecommender: 
    
    def __init__(self, dataframe, nx_graph_obj): 
        self.df = dataframe
        self.case_text = self.df.case_text 
        self.title_date = self.df.title_date
        
        self.all_citations= self.get_total_corpus_citations(self.case_text)
        
        self.g = nx_graph_obj        
        self.graph_dict = self.get_graph_dict(self.df, self.all_citations)
        self.weighted_edge_graph = self.build_weighted_edge_graph()
        # self.g and the weighted egde graph will become the same. 
        # just instantiated here to create the weighted edge graph
        
        ## Build Community/Node Dicts from graph 
        self.parts, self.check = self.build_node_community_dicts(self.g) 
        ## self.parts = {Node: Community}
        # EX: how to run search of community number: 
        """
        gr = GraphRecommender()
        search_query='Carpio v. Holder 01/13/2010'
        community_value = gr.parts[search_query]
        print(community_value)
        """
        ## self.check = {Community: Nodes}
        # EX: how to look up cases inside a community 
        """
        commuity_nodes = gr.check[community_value]
        print(len(commuity_nodes))
        print(commuity_nodes)        
        """
    
    def _get_citation(self, citation):
        """
        :param citation: string, a full citation
        :returns: string, a partial citation to serve as the nodes
        EX:
            Baker v. McCollan (1979)
            Graham v. Connor (1989)
            Austin v. Hamilton (1991)
        """
        cite = []
        for ind, word in enumerate(citation.split(' ')):
            if ind == 0 or ind == 1 or ind == 2:
                cite.append(word.strip(','))
            try:
                s = re.search(r"\d+(\.\d+)?", word)
                year = int(s.group(0))
                if year > 1750 and year <= 2018 and ind > (len(citation.split(' '))) * .80:
                    year = "(", str(year), ")"
                    year = "".join(year)
                    cite.append(year)
            except:
                continue
        return " ".join(cite)


    def _get_all_citations(self, document):
        """
        :param document: string, legal document with citations
        :returns: list, a list of all citations within the document
        """
        citations_objects = find_citation.get_citations(document, html=False)
        citations_strings = []
        for obj in citations_objects:
            citations_strings.append(str(obj))
        results = []
        for citation in citations_strings:
            results.append(self._get_citation(citation))
        return results

    def get_total_corpus_citations(self, data):
        """
        :param data: pandas series, all court document texts
        :return: list of lists, list of all document citation lists
        """
        return [self._get_all_citations(document) for document in data]

    def final_list_of_citations(self, all_citations):
        """
        :param all_citations: list of lists, each list has all the citations per document
        :returns: list of lists, updated lists: replaced "States" as plaintiff with "United States"

        EX:
            all_citations= "States v. Brown (2008)"
            returns = "United States v. Brown (2008)"
        """
        replace_words = {"states"}
        for first, lst in enumerate(all_citations):
            for second, citation in enumerate(lst):
                cite = citation.split(' ')[0]
                if cite.lower() in replace_words:
                    all_citations[first][second] = citation.replace(cite, "United States")
        # Updated Citations
        return all_citations

    def get_graph_dict(self, case_data, all_citations):
        """
        :param data: Pandas series, all the court case texts
        :param all_citations: list of lists, all the corpus citations
        :return: dict, key: court case, value: List of citations
        """
        graph_dict = {}
        fixed_citations = self.final_list_of_citations(all_citations)
        for ind, document in enumerate(case_data.case_text):
            graph_dict[case_data.title_date[ind]] = fixed_citations[ind]
        return graph_dict
    
    def _get_weights(self, list_a, list_b):
        """
        :param list_a: string, first node
        :param list_b: string, next node
        :returns: int, weights
        """
        return len(set(list_a) & set(list_b))
    
    
    def build_weighted_edge_graph(self):
        """
        This function is to create a weighted edge graph made of
        court cases (nodes) connected by shared citations (edge weights) 
        to be used to identify communities, and make recommendations 
        of similar cases per search result.
        :returns: None
        """
        for ind, (key, value) in enumerate(self.graph_dict.items()): 
            self.g.add_node(key)
            list_a = self.graph_dict[key]
            for inx, (other_nodes, other_values) in enumerate(self.graph_dict.items(), ind + 1): 
                list_b = self.graph_dict[other_nodes]   
                if list_a == list_b:
                    continue
                else:  
                    weight = self._get_weights(list_a, list_b)
                    self.g.add_edge(key, other_nodes, weight = weight)
        return self.g
    
    def build_node_community_dicts(self, g): 
        """
        :param g: networkx graph object
        :returns parts: dict, {Node: Community}; 
                check: dict, {Community: Node}
        """
        # parts = {Node: Community} 
        parts = community.best_partition(g, weight='weight')
        # check = {Community: Node} 
        check = {}
        for key, value in parts.items(): 
            if value not in check: 
                check[value] = [key]
            else: 
                check[value].append(key)
        return parts, check

    def get_recommendations(self, fitted_g, node_query, commuity_nodes, weights=False, top_n=5): 
        """
        :param G: a fitted network graph object
        :param node_query: string, used to locate a community.
        :param community_nodes: list of strings, of title_dates 
        :returns: a list, of ranked recommendations 
        """
        H = self.g.subgraph(commuity_nodes)
        edges = H.edges(data=True)

        ## list of 3-tuples; where 3rd item is dictionary {'weight': value}
        node_community = [tup for tup in edges if node_query in tup]
        sort = sorted(node_community, key=lambda x: x[2]['weight'])[::-1][:top_n]
        weighted_results_list = defaultdict(None)
        for tup in sort: 
            for item in tup: 
                if node_query not in item and type(item) != dict: 
                    weighted_results_list[item] = None
                elif type(item) == dict:
                    weighted_results_list[tup[1]] = item
                else: 
                    continue
        if weights: 
            results = {key: value['weight'] for key, value in weighted_results_list.items()} 
        else: 
            results = [res for res in weighted_results_list.keys()]
        return results 
        

In [3]:
import community

In [2]:
import networkx

In [4]:
community.best_partition

<function community.community_louvain.best_partition>